## 🔗 Open This Notebook in Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DavidLangworthy/ds4s/blob/master/days/day03/starter/day03_starter.ipynb)

# 🌫️ Day 3 – Pollution and Public Health
### Linking air-quality exposure to economic development

Today we connect environmental conditions to human outcomes. We'll pair fine particulate (PM₂.₅) exposure with GDP per capita to explore how pollution burdens shift with income. Expect rapid feedback loops after every transformation so you can debug before plotting.

### Data card — World Bank World Development Indicators
- **Source:** [World Bank – World Development Indicators](https://data.worldbank.org/)
- **Indicators used:** PM2.5 air pollution, mean annual exposure (EN.ATM.PM25.MC.M3) and GDP per capita (NY.GDP.PCAP.CD)
- **Temporal coverage:** 1990–2023 (annual; earlier years sparse)
- **Geographic coverage:** Countries and regional aggregates
- **Units:** PM₂.₅ in micrograms per cubic meter; GDP per capita in current US dollars
- **Processing notes:** Files include metadata header rows and many blank cells; we need to reshape from wide to tidy format.
- **Caveats:** GDP is nominal (not adjusted for purchasing power) and pollution exposure estimates carry uncertainty, especially in low-income regions.

### Preview: today's target chart
You'll produce a scatter plot with a fitted trend line, annotated to emphasize the inequity of pollution exposure across income levels.

In [ ]:
from pathlib import Path
from warnings import warn

import numpy as np
import pandas as pd
from urllib.error import HTTPError, URLError
from urllib.request import urlopen
import matplotlib.pyplot as plt
from IPython.display import Image, display

CANDIDATES = [Path.cwd(), *Path.cwd().parents]
for candidate in CANDIDATES:
    if (candidate / "data").exists():
        PROJECT_ROOT = candidate
        break
else:
    raise FileNotFoundError("Couldn't find the project root containing a data/ folder.")

DATA_DIR = PROJECT_ROOT / "data"
RAW_DATA_BASE = "https://raw.githubusercontent.com/DavidLangworthy/ds4s/master/data"
PLOTS_DIR = PROJECT_ROOT / "plots"
PLOTS_DIR.mkdir(exist_ok=True)

plt.rcParams.update({
    "figure.dpi": 110,
    "axes.titlesize": 16,
    "axes.labelsize": 12,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
    "axes.grid": True,
    "grid.alpha": 0.25,
})

COLORBLIND_FRIENDLY = ["#2E86AB", "#F18F01", "#A23B72", "#C73E1D", "#2EC4B6", "#33673B"]

if (PLOTS_DIR / "day03_solution_plot.png").exists():
    display(Image(filename=PLOTS_DIR / "day03_solution_plot.png", width=420))
else:
    print("Preview image not found; continue with the workflow.")


def load_data(csv_name: str, **read_kwargs) -> pd.DataFrame:
    path = DATA_DIR / csv_name
    if not path.exists():
        url = f"{RAW_DATA_BASE}/{csv_name}"
        print(f"Local file not found. Downloading {csv_name} from GitHub…")
        try:
            with urlopen(url, timeout=30) as response:
                data = response.read()
        except HTTPError as exc:
            raise FileNotFoundError(
                f"Could not retrieve {csv_name} from {url} (status {exc.code})."
            ) from exc
        except URLError as exc:
            raise FileNotFoundError(
                f"Could not retrieve {csv_name} from {url} ({exc.reason})."
            ) from exc
        path.parent.mkdir(parents=True, exist_ok=True)
        path.write_bytes(data)
    df = pd.read_csv(path, **read_kwargs)
    print(f"Loaded {csv_name} → {df.shape[0]:,} rows × {df.shape[1]} columns.")
    return df

def validate_columns(df: pd.DataFrame, required: list[str]) -> None:
    missing = [col for col in required if col not in df.columns]
    if missing:
        warn(f"Missing columns: {missing}")
    else:
        print("✅ Columns look good:", required)


def expect_rows_between(df: pd.DataFrame, lower: int, upper: int) -> None:
    rows = len(df)
    if rows < lower or rows > upper:
        warn(f"Row count {rows:,} outside the expected range ({lower:,}–{upper:,}).")
    else:
        print(f"✅ Row count within expected range ({rows:,}).")


def quick_diagnostics(df: pd.DataFrame, name: str = "DataFrame") -> None:
    print(f"--- Quick check for {name} ---")
    print("Shape:", df.shape)
    print("Columns:", list(df.columns))
    print("Missing values:", df.isna().sum())
    display(df.head())


def check_story_fields(**fields) -> None:
    empty = [key for key, value in fields.items() if not str(value).strip()]
    if empty:
        warn(f"These storytelling fields still need text: {', '.join(empty)}")
    else:
        print("✅ Story scaffolding complete. Ready to plot!")


def baseline_style():
    plt.style.use("seaborn-v0_8-whitegrid")
    plt.rcParams["axes.facecolor"] = "#F8FAFC"


def save_last_fig(filename: str) -> None:
    path = PLOTS_DIR / filename
    plt.savefig(path, dpi=300, bbox_inches="tight")
    print(f"Figure saved to {path.relative_to(PROJECT_ROOT)}")


## Step 1. Load and inspect the PM₂.₅ dataset
World Bank CSVs include metadata rows; we skip them and reshape later.

### Mini example: loading with UTF-8 safety
Here's a quick look at how `load_data` reads a CSV and reports its shape.

In [ ]:
import pandas as pd
demo = pd.DataFrame({
    "Country Name": ["Example"],
    "Country Code": ["EXM"],
    "1960": [55.0],
})
quick_diagnostics(demo, name="demo PM2.5 table")


In [ ]:
pm25_wide = load_data("pm25_exposure.csv", encoding="utf-8-sig")
pm25_wide = pm25_wide.drop(columns=[col for col in pm25_wide.columns if col.startswith("Unnamed")])
quick_diagnostics(pm25_wide, name="PM2.5 wide format")


✅ **Checkpoint:** You should see columns `Country Name`, `Country Code`, indicator metadata, followed by year columns (1960…2023).

## Step 2. Load GDP per capita data
We'll follow the same pattern so both indicators share the same shape.

### Mini example: aligning GDP data
The GDP file follows the same structure, so the same helper applies.

In [ ]:
import pandas as pd
demo_gdp = pd.DataFrame({
    "Country Name": ["Example"],
    "Country Code": ["EXM"],
    "1960": [1200.0],
})
quick_diagnostics(demo_gdp, name="demo GDP table")


In [ ]:
gdp_wide = load_data("gdp_per_country.csv", encoding="utf-8-sig")
gdp_wide = gdp_wide.drop(columns=[col for col in gdp_wide.columns if col.startswith("Unnamed")])
quick_diagnostics(gdp_wide, name="GDP wide format")


## Step 3. Tidy the datasets
Convert from wide to long format, keep country-level observations, and drop missing values.

### Mini example: melting wide to long
Practice melting a mini table before applying it to the full indicators.

In [ ]:
import pandas as pd
wide = pd.DataFrame({
    "Country Name": ["Example"],
    "Country Code": ["EXM"],
    "1990": [40.0],
    "1991": [41.0],
})
long = wide.melt(id_vars=["Country Name", "Country Code"], var_name="Year", value_name="value")
print(long)


In [ ]:
def tidy_indicator(df: pd.DataFrame, value_name: str) -> pd.DataFrame:
    id_vars = ["Country Name", "Country Code"]
    value_vars = [col for col in df.columns if col.isdigit()]
    tidy = (
        df[id_vars + value_vars]
        .melt(id_vars=id_vars, value_vars=value_vars, var_name="Year", value_name=value_name)
        .dropna(subset=[value_name])
    )
    tidy["Year"] = tidy["Year"].astype(int)
    tidy[value_name] = tidy[value_name].astype(float)
    return tidy

pm25_long = tidy_indicator(pm25_wide, "pm25")
gdp_long = tidy_indicator(gdp_wide, "gdp_per_capita")

expect_rows_between(pm25_long, 5_000, 20_000)
expect_rows_between(gdp_long, 5_000, 20_000)
quick_diagnostics(pm25_long.head(), name="PM2.5 tidy preview")


### Mini self-diagnostic
If the year column isn't numeric, check that you selected only digit-only column names before melting.

## Step 4. Join the indicators and filter to a storytelling year
We'll focus on 2022 (latest year with broad coverage) and drop aggregate regions so each dot represents a country.

### Mini example: merging indicators
See how an inner join on country and year keeps only aligned records.

In [ ]:
import pandas as pd
pm = pd.DataFrame({"Country Code": ["EXM"], "Year": [2022], "pm25": [35.2]})
gdp = pd.DataFrame({"Country Code": ["EXM"], "Year": [2022], "gdp_per_capita": [1500]})
merged = pm.merge(gdp, on=["Country Code", "Year"], how="inner")
print(merged)


In [ ]:
merged_all = pm25_long.merge(gdp_long, on=["Country Name", "Country Code", "Year"], how="inner")
available_years = merged_all['Year'].dropna().astype(int)
latest_year = int(available_years.max())
merged = merged_all.query("Year == @latest_year")

country_level = merged[~merged["Country Code"].str.contains("^X", na=False)]
country_level = country_level[country_level["Country Name"].str.contains("income", case=False) == False]

sample_size = min(len(country_level), 5)
quick_diagnostics(country_level.sample(sample_size, random_state=42) if sample_size else country_level, name="Joined sample")
print("Latest year available:", latest_year)
print("Countries in final sample:", country_level.shape[0])

✅ **Expectation check:** You should retain 150+ countries. If the sample is tiny, confirm the year choice exists in both datasets.

## Step 5. Compute reference lines and draft the story scaffold
We'll log-scale GDP for legibility, compute a trend line, and capture the narrative message.

### Mini example: computing log10 GDP and thresholds
Use simple numbers to preview the math before plotting.

In [ ]:
import numpy as np
sample_gdp = np.array([500, 2000, 10000])
log_gdp = np.log10(sample_gdp)
print("log10 values:", log_gdp)


In [ ]:
country_level["log_gdp"] = np.log10(country_level["gdp_per_capita"])
coeffs = np.polyfit(country_level["log_gdp"], country_level["pm25"], deg=1)
trend_line = np.poly1d(coeffs)

TITLE = "Air pollution exposure falls as income rises, but not fast enough"
SUBTITLE = f"PM₂.₅ exposure vs. GDP per capita for {latest_year}"
ANNOTATION = "Low-income countries breathe 2–3× the WHO guideline despite modest GDP gains."
SOURCE = "Source: World Bank World Development Indicators (EN.ATM.PM25.MC.M3; NY.GDP.PCAP.CD)"
UNITS = "Units: PM₂.₅ (μg/m³), GDP per capita (current USD; log scale)"

check_story_fields(
    TITLE=TITLE,
    SUBTITLE=SUBTITLE,
    ANNOTATION=ANNOTATION,
    SOURCE=SOURCE,
    UNITS=UNITS,
)


## Step 6. Plot the pollution-health scatter with a fitted trend
We'll use a log scale for GDP, draw the regression line, and annotate the inequity.

In [ ]:
baseline_style()
fig, ax = plt.subplots(figsize=(9, 6))

ax.scatter(
    country_level["gdp_per_capita"],
    country_level["pm25"],
    s=45,
    alpha=0.7,
    color=COLORBLIND_FRIENDLY[0],
    edgecolor="#0f172a",
    linewidth=0.4,
)

sorted_gdp = np.linspace(country_level["gdp_per_capita"].min(), country_level["gdp_per_capita"].max(), 200)
ax.plot(sorted_gdp, trend_line(np.log10(sorted_gdp)), color=COLORBLIND_FRIENDLY[1], linewidth=2.2, label="Trend line")

ax.set_xscale("log")
ax.set_xlabel("GDP per capita (current US$; log scale)")
ax.set_ylabel("PM₂.₅ exposure (μg/m³)")

title_text = TITLE + "\n" + SUBTITLE
ax.set_title(title_text, loc="left", pad=18)
ax.axhline(5, color="#EF4444", linestyle="--", linewidth=1, label="WHO guideline (5 μg/m³)")
ax.legend(frameon=False, loc="upper right")

worst_case = country_level.sort_values("pm25", ascending=False).iloc[0]
ax.annotate(
    ANNOTATION,
    xy=(worst_case["gdp_per_capita"], worst_case["pm25"]),
    xytext=(worst_case["gdp_per_capita"] * 3, worst_case["pm25"] + 10),
    arrowprops=dict(arrowstyle="->", color="#475569"),
    fontsize=11,
    color="#111827",
)

caption_text = SOURCE + "\n" + "Claim → Evidence → Visual → Takeaway: Income growth helps, yet many countries stay far above the WHO safety line."
fig.text(0.01, -0.08, caption_text, fontsize=10)
plt.show()


### Accessibility checklist
- Dot colors and trend line use contrasting, colorblind-friendly hues.
- Log axis labeled clearly to avoid misinterpretation.
- Annotation highlights interpretation rather than restating the obvious.
- WHO guideline line provides context without dual axes.

In [ ]:
save_last_fig("day03_solution_plot.png")


## Step 7. Reflect on limitations and ethics
- PM₂.₅ estimates rely on satellite models and sparse ground monitors; uncertainties are highest in low-income regions.
- GDP per capita doesn't capture inequality within countries; pollution burdens can fall disproportionately on marginalized communities.
- Encourage learners to surface questions they cannot answer with this scatter alone (e.g., policy causes, health outcomes).
- Invite a discussion on ethical framing: avoid implying pollution is an acceptable trade-off for growth.